In [ ]:
# üì¶ Install the dependencies
!pip install groq langchain faiss-cpu gradio tiktoken sentence-transformers transformers langdetect langchain-community langchain-groq pypdf

# üìö Imports
import os
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader

# üîê Set up Groq API Key
os.environ["GROQ_API_KEY"] = "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"

# üß† LLM setup
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-8b-8192")

# üìÑ Load and process legal document
def load_legal_doc(file_obj):
    try:
        loader = PyPDFLoader(file_obj.name)
        pages = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = splitter.split_documents(pages)
        embeddings = HuggingFaceEmbeddings()
        vectorstore = FAISS.from_documents(docs, embeddings)
        return vectorstore
    except Exception as e:
        raise ValueError(f"Failed to process PDF: {str(e)}")

# üîÅ Session state
qa_chain = None
vectorstore = None

# üí¨ Main legal chatbot function
def legal_chat(user_input, file, feature):
    global qa_chain, vectorstore

    if feature == "Indian Legal Judgment Advisor":
        prompt = f"""
You are a legal advisor specializing in Indian Penal Code (IPC). Based on the situation described below, respond briefly with:

1. ‚úÖ Verdict: Guilty or Not Guilty
2. üìú IPC Section(s): Mention only applicable section numbers
3. üß† Reason: 1-2 line simple explanation
4. ‚öñÔ∏è Max Punishment: Just the maximum punishment allowed

Scenario: {user_input}

Respond only in bullet points. Do not return extra metadata or explanations.
"""
        return llm.invoke(prompt).content  # ‚úÖ Clean result only

    if file is None or file.name == "":
        return "‚ö†Ô∏è Please upload a valid legal PDF document."

    try:
        if vectorstore is None:
            vectorstore = load_legal_doc(file)
            retriever = vectorstore.as_retriever()
            qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

        if feature == "Standard QA":
            return qa_chain.run(user_input)

        elif feature == "Legal Template Generator":
            return llm.invoke(f"üìë Create a legal template for: {user_input}").content

        elif feature == "Legal Reasoning Chain":
            return llm.invoke(f"üîç Explain the legal reasoning step-by-step for: {user_input}").content

        elif feature == "Legal Risk Detector":
            return llm.invoke(f"‚ö†Ô∏è Analyze this document and detect risky clauses about: {user_input}").content

        else:
            return "‚ö†Ô∏è Unknown feature selected."

    except Exception as e:
        return f"‚ùå Error: {str(e)}"


# üé® Gradio UI
with gr.Blocks() as app:
    gr.Markdown("## üß† AI Lawyer Assistant (Upload Legal PDFs & Ask Questions)")

    with gr.Row():
        feature_dropdown = gr.Dropdown(label="üîß Feature", choices=[
            "Standard QA",
            "Legal Template Generator",
            "Legal Reasoning Chain",
            "Legal Risk Detector",
            "Indian Legal Judgment Advisor"
        ], value="Standard QA")

    file_input = gr.File(label="üìÑ Upload Legal PDF", file_types=[".pdf"], visible=True)
    user_input = gr.Textbox(label="üí¨ Ask your legal question or describe a situation")
    output = gr.Textbox(label="üßæ AI Response")
    send_btn = gr.Button("üöÄ Get Answer")

    # üß† Dynamically show/hide PDF upload based on selected feature
    def toggle_file_visibility(feature):
        return gr.update(visible=(feature != "Indian Legal Judgment Advisor"))

    feature_dropdown.change(fn=toggle_file_visibility, inputs=feature_dropdown, outputs=file_input)
    send_btn.click(fn=legal_chat, inputs=[user_input, file_input, feature_dropdown], outputs=[output])

app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c6826cd733ce6836c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
